In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi

# Nome das classes (substitua pelos nomes das suas classes)
classes = ['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4', 'Classe 5', 'Classe 6', 'Classe 7']

# Dados de exemplo: Precision, Recall, e F1 para cada modelo e classe
# Cada linha representa uma métrica para um modelo
# Ajuste esses valores para os seus resultados reais
precision_data = [
    [0.8, 0.7, 0.6, 0.9, 0.5, 0.4, 0.7],  # Modelo 1 (ResNet-101)
    [0.75, 0.65, 0.55, 0.85, 0.6, 0.5, 0.65],  # Modelo 2 (ResNet-50)
    [0.7, 0.6, 0.5, 0.8, 0.55, 0.45, 0.6],  # Modelo 3 (GoogleNet)
    [0.65, 0.55, 0.45, 0.75, 0.5, 0.4, 0.55]  # Modelo 4 (VGG-16)
]

# Criação dos ângulos para o radar
num_vars = len(classes)
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]  # Fechar o gráfico

# Função para criar gráficos de radar
def plot_radar(data, title, labels):
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

    # Desenhar uma linha e preencher para cada modelo
    for idx, values in enumerate(data):
        values += values[:1]  # Fechar o gráfico
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=labels[idx])
        ax.fill(angles, values, alpha=0.1)

    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(classes, fontsize=10)
    plt.title(title, size=15, color='black', y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    plt.show()

# Exibir gráficos de radar para precisão, recall e F1
plot_radar(precision_data, 'Precision - Modelos', ['ResNet-101', 'ResNet-50', 'GoogleNet', 'VGG-16'])
# Repita para os dados de recall e F1 se tiver


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR, SequentialLR

import torchvision
from torchvision import models
from torchvision.models.convnext import (convnext_tiny, ConvNeXt_Tiny_Weights,
                                         convnext_small, ConvNeXt_Small_Weights,
                                         convnext_base, ConvNeXt_Base_Weights,
                                         convnext_large, ConvNeXt_Large_Weights)
from torchvision.models.swin_transformer import (swin_t, Swin_T_Weights,
                                                 swin_s, Swin_S_Weights,
                                                 swin_b, Swin_B_Weights)
import torchmetrics
from torchmetrics import Accuracy

import pytorch_lightning as pl

import config
from compact_transform.src import cct_14_7x2_224, cct_14_7x2_384, cct_14_7x2_384_fl



class CustomModel(pl.LightningModule):
    def __init__(self, tmodel, epochs, learning_rate, scale_factor,
                 drop_path_rate, num_classes, label_smoothing, optimizer_momentum):
        
        super(CustomModel, self).__init__()

        self.epochs = epochs
        self.learning_rate = learning_rate
        self.scale_factor = scale_factor
        self.drop_path_rate = drop_path_rate
        self.num_classes = num_classes
        self.label_smoothing = label_smoothing
        self.optimizer_momentum = optimizer_momentum
        self.fn_loss = nn.CrossEntropyLoss(label_smoothing=self.label_smoothing)
        
        # Métricas
        self.train_accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

        self.model = models.convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT, 
                                        drop_path_rate=self.drop_path_rate)
        self.sequential_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LayerNorm(768, eps=1e-6, elementwise_affine=True),
            nn.Linear(in_features=768, out_features=self.num_classes, bias=True)
        )
        self.model.classifier = self.sequential_layers

    def forward(self, x):
        return self.model(x)


    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)

        # Calcular a precisão
        self.train_accuracy(preds, labels)
        
        # Logar a perda e a acurácia
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_accuracy', self.train_accuracy, prog_bar=True, on_step=False, on_epoch=True)
        
        # Retornar a perda para o processamento posterior
        return {'loss': loss}
    

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)
        
        # Calcular a precisão para validação
        self.val_accuracy(preds, labels)
        
        # Logar a perda e a acurácia no conjunto de validação
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        self.log('val_accuracy', self.val_accuracy, prog_bar=True, on_epoch=True)
        
        # Retornar a perda e a acurácia
        return {'val_loss': loss}
    
    # def on_train_epoch_end(self):
    #     # Acessar a perda média do treino automaticamente através do logger
    #     avg_loss = self.trainer.callback_metrics['train_loss']

    #     # Imprimir a perda média de treino
    #     print(f'Loss médio do treino na época: {avg_loss:.4f}')
    
    # def on_validation_epoch_end(self):
    #     # Acessar a perda média da validação automaticamente através do logger
    #     avg_val_loss = self.trainer.callback_metrics['val_loss']

    #     # Imprimir a perda média da validação
    #     print(f'Loss médio da validação na época: {avg_val_loss:.4f}')


    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)

        # Calcular a precisão
        self.test_accuracy(preds, labels)   
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_accuracy, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        # Definir o otimizador com os grupos de parâmetros
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, betas = self.optimizer_momentum)

        # Definir o scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.epochs)

        # Retornar o otimizador e o scheduler
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'epoch',  # Step the scheduler per epoch
                'monitor': 'val_loss',  # Optional, monitor val_loss (useful for other schedulers)
                'frequency': 1,  # Apply the scheduler every epoch
            }
        }

In [ ]:
    def forward(self, x):
        return self.model(x)


    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)

        # Calcular a precisão
        self.train_accuracy(preds, labels)
        
        # Logar a perda e a acurácia
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_accuracy', self.train_accuracy, prog_bar=True, on_step=False, on_epoch=True)
        
        # Retornar a perda para o processamento posterior
        return {'loss': loss}
    

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)
        
        # Calcular a precisão para validação
        self.val_accuracy(preds, labels)
        
        # Logar a perda e a acurácia no conjunto de validação
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        self.log('val_accuracy', self.val_accuracy, prog_bar=True, on_epoch=True)
        
        # Retornar a perda e a acurácia
        return {'val_loss': loss}
    
    # def on_train_epoch_end(self):
    #     # Acessar a perda média do treino automaticamente através do logger
    #     avg_loss = self.trainer.callback_metrics['train_loss']

    #     # Imprimir a perda média de treino
    #     print(f'Loss médio do treino na época: {avg_loss:.4f}')
    
    # def on_validation_epoch_end(self):
    #     # Acessar a perda média da validação automaticamente através do logger
    #     avg_val_loss = self.trainer.callback_metrics['val_loss']

    #     # Imprimir a perda média da validação
    #     print(f'Loss médio da validação na época: {avg_val_loss:.4f}')


    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = self.fn_loss(logits, labels)
        preds = torch.argmax(logits, 1)

        # Calcular a precisão
        self.test_accuracy(preds, labels)   
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_accuracy, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        # Definir o otimizador com os grupos de parâmetros
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, betas = self.optimizer_momentum)

        # Definir o scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.epochs)

        # Retornar o otimizador e o scheduler
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'epoch',  # Step the scheduler per epoch
                'monitor': 'val_loss',  # Optional, monitor val_loss (useful for other schedulers)
                'frequency': 1,  # Apply the scheduler every epoch
            }
        }

In [ ]:
import yaml
def load_hyperparameters(file_path):
    with open(file_path, 'r') as file:
        hyperparams = yaml.safe_load(file)  # Carregar o YAML
    return hyperparams

hyperparams = load_hyperparameters('config.yaml')

In [1]:
from torchvision import models
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
model = models.swin_v2_t(weights=Swin_V2_T_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/swin_v2_t-b137f0e2.pth" to /home/rodrigoduarte/.cache/torch/hub/checkpoints/swin_v2_t-b137f0e2.pth
100%|██████████| 109M/109M [00:11<00:00, 10.3MB/s] 


In [2]:
model

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='

In [1]:
import os

def count_classes(dataset_dir):
    # Lista de subdiretórios dentro do diretório do dataset
    classes = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    num_classes = len(classes)
    
    print(f"Número de classes: {num_classes}")
    print("Classes:", classes)
    
    return num_classes, classes

# Exemplo de uso
train_dir = "/home/rodrigoduarte/Documentos/projeto/imagens/flavia/train"  # Substitua pelo caminho para o diretório de treino
count_classes(train_dir)


Número de classes: 32
Classes: ['Anhui Barberry', 'Chinese cinnamon', 'trident maple', 'Chinese redbud', 'oleander', 'Chinese horse chestnut', 'Chinese Toon', 'wintersweet', 'deodar', 'Japan Arrowwood', 'Glossy Privet', 'camphortree', 'Japanese cheesewood', 'pubescent bamboo', 'peach', 'true indigo', 'sweet osmanthus', 'Chinese tulip tree', 'Japanese Flowering Cherry', 'yew plum pine', 'goldenrain tree', 'Big-fruited Holly', "Beale's barberry", 'Japanese maple', 'ginkgo, maidenhair tree', 'tangerine', 'castor aralia', 'Canadian poplar', 'Crape myrtle, Crepe myrtle', 'southern magnolia', 'Nanmu', 'Ford Woodlotus']


(32,
 ['Anhui Barberry',
  'Chinese cinnamon',
  'trident maple',
  'Chinese redbud',
  'oleander',
  'Chinese horse chestnut',
  'Chinese Toon',
  'wintersweet',
  'deodar',
  'Japan Arrowwood',
  'Glossy Privet',
  'camphortree',
  'Japanese cheesewood',
  'pubescent bamboo',
  'peach',
  'true indigo',
  'sweet osmanthus',
  'Chinese tulip tree',
  'Japanese Flowering Cherry',
  'yew plum pine',
  'goldenrain tree',
  'Big-fruited Holly',
  "Beale's barberry",
  'Japanese maple',
  'ginkgo, maidenhair tree',
  'tangerine',
  'castor aralia',
  'Canadian poplar',
  'Crape myrtle, Crepe myrtle',
  'southern magnolia',
  'Nanmu',
  'Ford Woodlotus'])